In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

data_dir = './data/'
file = 'userid-timestamp-artid-artname-traid-traname.tsv'

In [3]:
df = pd.read_csv(data_dir + file, delimiter="\t", header=None, error_bad_lines=False)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [3]:
df.describe()

,0,1,2,3,4,5
count,19098853,19098853,18498005,19098853,16936134,19098841
unique,992,17454730,107295,173921,960402,1083471
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,183103,248,115099,115099,3991,17561


In [4]:
drop_na_df = df.dropna(axis=0, how='any')

In [5]:
drop_na_df.head(5)

,0,1,2,3,4,5
10,user_000001,2009-05-04T13:06:09Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,f7c1f8f8-b935-45ed-8fc8-7def69d92a10,The Last Emperor (Theme)
12,user_000001,2009-05-04T12:55:34Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,475d4e50-cebb-4cd0-8cd4-c3df97987962,Tibetan Dance (Version)
14,user_000001,2009-05-03T15:48:25Z,ba2f4f3b-0293-4bc8-bb94-2f73b5207343,Underworld,dc394163-2b78-4b56-94e4-658597a29ef8,"Boy, Boy, Boy (Switch Remix)"
15,user_000001,2009-05-03T15:37:56Z,ba2f4f3b-0293-4bc8-bb94-2f73b5207343,Underworld,340d9a0b-9a43-4098-b116-9f79811bd508,Crocodile (Innervisions Orchestra Mix)
16,user_000001,2009-05-03T15:14:53Z,a16e47f5-aa54-47fe-87e4-bb8af91a9fdd,Ennio Morricone,0b04407b-f517-4e00-9e6a-494795efc73e,Ninna Nanna In Blu (Raw Deal Remix)


In [6]:
drop_na_df.describe()

,0,1,2,3,4,5
count,16936134,16936134,16936134,16936134,16936134,16936134
unique,992,15631466,83905,81751,960402,693231
top,user_000949,2009-02-26T21:29:15Z,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,db16d0b3-b8ce-4aa8-a11a-e4d53cc7f8a6,Intro
freq,172042,193,111488,111488,3991,14908


In [2]:
file_name = 'original_set'
#drop_na_df.to_pickle(data_dir + file_name)
drop_na_df = pd.read_pickle(data_dir + file_name)

In [5]:
drop_na_df.tail()

,0,1,2,3,4,5
19098848,user_001000,2008-01-27T22:02:35Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,a490cabc-1e5c-4807-86c7-740c31a50009,Please Be Patient With Me
19098849,user_001000,2008-01-27T21:56:52Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,3e92e447-9e1f-440d-bc00-6734469880c5,Shake It Off
19098850,user_001000,2008-01-27T21:52:36Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,93d044e6-1bbb-46a6-ac8e-283382a89e6f,Side With The Seeds
19098851,user_001000,2008-01-27T21:49:12Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,5ac4386f-6146-4389-a762-4b43f362d2c8,Sky Blue Sky
19098852,user_001000,2008-01-27T21:43:14Z,9e53f84d-ef44-4c16-9677-5fd4d78cbd7d,Wilco,3acc99bc-a349-420f-ad28-7095eb3533c9,Impossible Germany


In [3]:
most_close_time = datetime.strptime("2009-05-05 23:59:59", "%Y-%m-%d %H:%M:%S")
print(most_close_time)
most_early_time = most_close_time - timedelta(days=180)
print(most_early_time)

2009-05-05 23:59:59
2008-11-06 23:59:59


In [4]:
temp_df = drop_na_df.copy()
f = lambda x: np.NaN if datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ") < most_early_time else x
temp_df.iloc[:,1:2] = drop_na_df.iloc[:,1:2].applymap(f)

split_date_df = temp_df.dropna(axis=0, how='any')
split_date_df

,0,1,2,3,4,5
10,user_000001,2009-05-04T13:06:09Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,f7c1f8f8-b935-45ed-8fc8-7def69d92a10,The Last Emperor (Theme)
12,user_000001,2009-05-04T12:55:34Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,475d4e50-cebb-4cd0-8cd4-c3df97987962,Tibetan Dance (Version)
14,user_000001,2009-05-03T15:48:25Z,ba2f4f3b-0293-4bc8-bb94-2f73b5207343,Underworld,dc394163-2b78-4b56-94e4-658597a29ef8,"Boy, Boy, Boy (Switch Remix)"
15,user_000001,2009-05-03T15:37:56Z,ba2f4f3b-0293-4bc8-bb94-2f73b5207343,Underworld,340d9a0b-9a43-4098-b116-9f79811bd508,Crocodile (Innervisions Orchestra Mix)
16,user_000001,2009-05-03T15:14:53Z,a16e47f5-aa54-47fe-87e4-bb8af91a9fdd,Ennio Morricone,0b04407b-f517-4e00-9e6a-494795efc73e,Ninna Nanna In Blu (Raw Deal Remix)
17,user_000001,2009-05-03T15:10:18Z,463a94f1-2713-40b1-9c88-dcc9c0170cae,Minus 8,4e78efc4-e545-47af-9617-05ff816d86e2,Elysian Fields
18,user_000001,2009-05-03T15:04:31Z,ad0811ea-e213-451d-b22f-fa1a7f9e0226,Beanfield,fb51d2c4-cc69-4128-92f5-77ec38d66859,Planetary Deadlock
19,user_000001,2009-05-03T14:56:25Z,309e2dfc-678e-4d09-a7a4-8eab9525b669,Dj Linus,4277434f-e3c2-41ae-9ce3-23fd157f9347,Good Morning Love Coffee Is Ready
20,user_000001,2009-05-03T14:50:51Z,6f3d4a7b-45b2-4c08-9306-8d271e92cb4f,Alif Tree,1151b040-8022-4965-96d2-8a4605ce456c,Deadly Species
21,user_000001,2009-05-03T14:46:29Z,463a94f1-2713-40b1-9c88-dcc9c0170cae,Minus 8,f78c95a8-9256-4757-9a9f-213df5c6854e,Cold Fusion


In [5]:
file_name = 'split_date_set'
split_date_df.to_pickle(data_dir + file_name)